<a href="https://colab.research.google.com/github/sjbaek12/sjbaek12.github.io/blob/master/%EC%95%99%EC%83%81%EB%B8%94_for_CNN_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

순환신경망과 합성곱 신경망의 앙상블 테스트

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb

In [8]:
(x_train_all, y_train_all), (x_test, y_test) = imdb.load_data(skip_top=20, num_words=100)

from tensorflow.keras.preprocessing import sequence

max_features = 1000
max_len=100
x_train_seq = sequence.pad_sequences(x_train_all, maxlen=max_len)

print(x_train_seq.shape)

(25000, 100)


In [30]:
from keras.models import Model
from keras import layers
from keras import Input
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, SimpleRNN, Average 

테스트를 위해서 Embedding층으로만 구성된 모델을 만들고 실행시켜본 예이다.
먼저 embedding 층을 모델링한다.

In [13]:
input_tensor = Input(shape=(100,)) # shape=(input_dim,)
output_tensor = Embedding(500, 128)(input_tensor)  # 500개 단어를 크기 128개의 벡터로 표현하면 64000개의 파라미터가 나온다

model_emb = Model(input_tensor, output_tensor)
model_emb.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 100, 128)          64000     
Total params: 64,000
Trainable params: 64,000
Non-trainable params: 0
_________________________________________________________________


다음으로 Embedding 모델을 컴파일 한다

In [17]:
model_emb.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [21]:
x_train_seq.shape

(25000, 100)

실제 데이터를 넣어서 embedding된 벡터들이 어떻게 구성되었는지 본 예이다. 여기서는 model.predict를 사용하면 된다. x_train_seq 전체가 하나의 데이터이기 때문에 전체를 예측하고, 그 다음 샘플[0]가 어떻게 구성되는지 본다

In [25]:
model_emb.predict(x_train_seq)[0].shape

(100, 128)

다음은 CNN을 함수형으로 모델링 한 것이다. Conv1D를 사용해서 문장의 극성을 평가하는 모델이다.

In [26]:
input_tensor = Input(shape=(100,)) # shape=(input_dim,)

def conv_pool_cnn(model_input):
  x = Embedding(500, 128)(model_input)
  x = Conv1D(32, 7, activation = 'relu')(x)
  x = MaxPooling1D(5)(x)
  x = Conv1D(32, 6, activation='relu')(x)
  x = GlobalMaxPooling1D()(x)
  x = Dense(1, activation = 'sigmoid')(x)

  model = Model(model_input, x, name='conv_pool_cnn')

  return model



model = conv_pool_cnn(input_tensor)
model.summary()

Model: "conv_pool_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 100, 128)          64000     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 94, 32)            28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 18, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 32)            6176      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)               

In [32]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
history = model.fit(x_train_seq, y_train_all, epochs=10, batch_size=128)

Epoch 1/10
25000/25000 [==============================] - 19s 767us/step - loss: 0.6437 - acc: 0.6119
Epoch 2/10
25000/25000 [==============================] - 19s 752us/step - loss: 0.6033 - acc: 0.6670
Epoch 3/10
25000/25000 [==============================] - 19s 754us/step - loss: 0.5909 - acc: 0.6792
Epoch 4/10
25000/25000 [==============================] - 19s 754us/step - loss: 0.5794 - acc: 0.6903
Epoch 5/10
25000/25000 [==============================] - 19s 754us/step - loss: 0.5695 - acc: 0.6980
Epoch 6/10
25000/25000 [==============================] - 21s 841us/step - loss: 0.5579 - acc: 0.7086
Epoch 7/10
25000/25000 [==============================] - 19s 761us/step - loss: 0.5468 - acc: 0.7168
Epoch 8/10
25000/25000 [==============================] - 19s 757us/step - loss: 0.5344 - acc: 0.7286
Epoch 9/10
25000/25000 [==============================] - 19s 755us/step - loss: 0.5215 - acc: 0.7368
Epoch 10/10
25000/25000 [==============================] - 19s 756us/step - loss: 

다음은 RNN을 함수형으로 모델링한 것이다. 다음은 전형적인 SimpleRNN을 이용하여 문장의 극성을 분류하는 모델이다.

In [36]:
def rnn_pool_cnn(model_input):
  x = Embedding(500, 128)(model_input)
  x = SimpleRNN(20)(x)
  x = Dense(1, activation='sigmoid')(x)
  
  model = Model(model_input, x, name='rnn_pool_cnn')

  return model

In [37]:
model1 = rnn_pool_cnn(input_tensor)
model1.summary()

Model: "rnn_pool_cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 100, 128)          64000     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 20)                2980      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 67,001
Trainable params: 67,001
Non-trainable params: 0
_________________________________________________________________


In [38]:
model1.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['acc'])
history1 = model1.fit(x_train_seq, y_train_all, epochs=10, batch_size=128)

Epoch 1/10
25000/25000 [==============================] - 8s 324us/step - loss: 0.6884 - acc: 0.5352
Epoch 2/10
25000/25000 [==============================] - 8s 315us/step - loss: 0.6700 - acc: 0.5858
Epoch 3/10
25000/25000 [==============================] - 8s 320us/step - loss: 0.6707 - acc: 0.5784
Epoch 4/10
25000/25000 [==============================] - 8s 325us/step - loss: 0.6681 - acc: 0.5883
Epoch 5/10
25000/25000 [==============================] - 8s 317us/step - loss: 0.6538 - acc: 0.6108
Epoch 6/10
25000/25000 [==============================] - 8s 319us/step - loss: 0.6498 - acc: 0.6246
Epoch 7/10
25000/25000 [==============================] - 8s 319us/step - loss: 0.6500 - acc: 0.6328
Epoch 8/10
25000/25000 [==============================] - 8s 320us/step - loss: 0.6213 - acc: 0.6628
Epoch 9/10
25000/25000 [==============================] - 8s 319us/step - loss: 0.6181 - acc: 0.6598
Epoch 10/10
25000/25000 [==============================] - 8s 317us/step - loss: 0.6063 - a

함수로 정의한 각자의 모델을 앙상블 모델로 결합하는 단계이다.

In [51]:
model = conv_pool_cnn(input_tensor)
model1 = rnn_pool_cnn(input_tensor)

#conv_pool_cnn_model.load_weights('weights/conv_pool_cnn.29-0.10.hdf5')
##all_cnn_model.load_weights('weights/all_cnn.30-0.08.hdf5')
#nin_cnn_model.load_weights('weights/nin_cnn.30-0.93.hdf5')

models = [model, model1]

In [40]:
def ensemble(models, model_input):
    
    outputs = [model.outputs[0] for model in models] # 각 모델의 output을 다음 Average() 층의 Input data로 투입한다.
    y = Average()(outputs)
    model = Model(model_input, y, name='ensemble') # 여기서 앙상블 Average()층의 output은 y이다.
    
    return model

ensemble_model = ensemble(models, input_tensor)

In [41]:
ensemble_model.summary()

Model: "ensemble"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 100, 128)     64000       input_6[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 94, 32)       28704       embedding_9[0][0]                
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, 18, 32)       0           conv1d_3[0][0]                   
___________________________________________________________________________________________

In [42]:
ensemble_model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['acc'])

In [43]:
history = ensemble_model.fit(x_train_seq, y_train_all, epochs=10, batch_size=128)

Epoch 1/10
25000/25000 [==============================] - 26s 1ms/step - loss: 0.6607 - acc: 0.5938
Epoch 2/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.6013 - acc: 0.6778
Epoch 3/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.5817 - acc: 0.6963
Epoch 4/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.5682 - acc: 0.7081
Epoch 5/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.5569 - acc: 0.7167
Epoch 6/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.5459 - acc: 0.7255
Epoch 7/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.5358 - acc: 0.7336
Epoch 8/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.5249 - acc: 0.7430
Epoch 9/10
25000/25000 [==============================] - 25s 1ms/step - loss: 0.5147 - acc: 0.7521
Epoch 10/10
25000/25000 [==============================] - 26s 1ms/step - loss: 0.5029 - acc: 0.7587

In [44]:
print(history.history['acc'])

[0.59376, 0.67776, 0.69632, 0.70808, 0.71668, 0.72548, 0.73364, 0.74304, 0.75208, 0.75872]


In [47]:
ensemble_model.predict(x_train_seq)[0:9] # 훈련 데이터의 첫번째 예측치이다.

array([[0.36376274],
       [0.2082777 ],
       [0.13261694],
       [0.3404683 ],
       [0.24681291],
       [0.60040665],
       [0.75392747],
       [0.43278816],
       [0.79250693]], dtype=float32)

In [48]:
y_train_all[0:9]

array([1, 0, 0, 1, 0, 0, 1, 0, 1])

이제 앙상블 모델을 다른 형태로 만들어 본다.

In [101]:
(x_train_all, y_train_all), (x_test, y_test) = imdb.load_data(skip_top=20, num_words=100)

from tensorflow.keras.preprocessing import sequence

max_features = 1000
max_len=100
x_train_seq1 = sequence.pad_sequences(x_train_all, maxlen=max_len)
x_train_seq2 = sequence.pad_sequences(x_train_all, maxlen=max_len)


print(x_train_seq1.shape)

(25000, 100)


In [52]:
from keras.layers.merge import concatenate

In [81]:
input_tensor1 = Input(shape=(100,)) # shape=(input_dim,)
h_conv = Embedding(500, 128)(input_tensor1)
h_conv = Conv1D(32, 7, activation = 'relu')(h_conv)
h_conv = MaxPooling1D(5)(h_conv)
h_conv = Conv1D(32, 6, activation='relu')(h_conv)
h_conv = GlobalMaxPooling1D()(h_conv)
o_conv = Dense(1, activation = 'sigmoid')(h_conv)

input_tensor2 = Input(shape=(100,)) # shape=(input_dim,)
h_rnn = Embedding(500, 128)(input_tensor2)
h_rnn = SimpleRNN(20)(h_rnn)
o_rnn = Dense(1, activation='sigmoid')(h_rnn)

merged_model = concatenate([o_conv, o_rnn]) # concatenate는 이어붙이기를 의미한다.

output = Dense(1,activation='sigmoid')(merged_model)

model_ensem_con = Model(inputs=[input_tensor1, input_tensor2], outputs=output)

In [82]:
model_ensem_con.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_23 (Embedding)        (None, 100, 128)     64000       input_16[0][0]                   
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 94, 32)       28704       embedding_23[0][0]               
__________________________________________________________________________________________________
max_pooling1d_9 (MaxPooling1D)  (None, 18, 32)       0           conv1d_17[0][0]                  
____________________________________________________________________________________________

In [99]:
model_ensem_con.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='binary_crossentropy', metrics=['acc'])

In [102]:
model_ensem_con.fit([x_train_seq1, x_train_seq2], y_train_all, epochs=10, batch_size=128)

Epoch 1/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6723 - acc: 0.5984
Epoch 2/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6681 - acc: 0.6046
Epoch 3/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6633 - acc: 0.6155
Epoch 4/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6655 - acc: 0.6134
Epoch 5/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6623 - acc: 0.6198
Epoch 6/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6631 - acc: 0.6161
Epoch 7/10
25000/25000 [==============================] - 30s 1ms/step - loss: 0.6617 - acc: 0.6190
Epoch 8/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6605 - acc: 0.6232
Epoch 9/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6617 - acc: 0.6197
Epoch 10/10
25000/25000 [==============================] - 29s 1ms/step - loss: 0.6596 - acc: 0.6221

In [119]:
model_ensem_con.predict([x_train_seq1, x_train_seq2])[0][0]

0.42216823

In [121]:
y_train_all[0]

1

In [124]:
con = 0

for i in range(30):
  a = model_ensem_con.predict([x_train_seq1, x_train_seq2])[i]
  if a >= 0.5:
    a = 1
  else:
    a = 0
  if a == y_train_all[i]:
    con = con + 1
  print(con/30)



0.0
0.03333333333333333
0.06666666666666667
0.06666666666666667
0.1
0.1
0.13333333333333333
0.16666666666666666
0.16666666666666666
0.2
0.23333333333333334
0.23333333333333334
0.26666666666666666
0.26666666666666666
0.26666666666666666
0.3
0.3333333333333333
0.3333333333333333
0.36666666666666664
0.36666666666666664
0.4
0.43333333333333335
0.4666666666666667
0.4666666666666667
0.5
0.5
0.5
0.5333333333333333
0.5666666666666667
0.6
